In [1]:
library(tidyverse)
library(caret)

── Attaching packages ───────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ──────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [2]:
setwd("~/Repositories/AirQualityCOVID/")

source("src/Cross-validation/useData.R")

In [3]:
sites.lv <- read.csv("data/Curation/checked_AQ.csv") %>%
            filter(#Municipio %in% c("Madrid", "Gijón"),
                   Pollutant == "o3") 

In [4]:
train.df <- open.data(sites = sites.lv$site, 
                      variable = c("o3"),
                      start_dt = lubridate::ymd("2013-01-01"), 
                      end_dt = lubridate::ymd("2020-01-01"),
                      airQuality.fl = "data/all/data_AQ.rda", 
                      meteo.fl = "data/all/meteorology.rda"
                     )

In [5]:
test.df <- open.data(sites = sites.lv$site, 
                     variable = c("o3"),
                     start_dt = lubridate::ymd("2019-12-31"), 
                     end_dt = lubridate::ymd("2021-01-01"),
                     airQuality.fl = "data/all/data_AQ.rda", 
                     meteo.fl = "data/all/meteorology.rda"
                    )

In [6]:
results <- data.frame()

for (nm in names(train.df)) {
    df.train <- train.df[[nm]] %>%
            add.yesterday.meteo(n.days=2) %>%
            na.omit() %>%
            select(-date)
    
    df.test <- test.df[[nm]] %>%
            add.yesterday.meteo(n.days=2) %>%
            na.omit()
    
    if (nrow(df.train) == 0) {
        next()
    }
    
    model <- train(o3 ~.,
               data=df.train,
               method="lm"
              )
    
    pred <- downscaleR:::eqm(df.train$o3,
                             predict(model),
                             predict(model, 
                                     newdata = df.test),
                             n.quantile=99,
                             precip=FALSE, pr.threshold=0,
                             extrapolation=""
                            )
    results <- rbind(results,
                     data.frame(site = nm,
                                date = df.test$date,
                                obs = df.test$o3,
                                pred = pred
                               ))
}

In [7]:
results$diff <- 100 *(results$obs - results$pred) / results$pred

In [8]:
head(results)

,site,date,obs,pred,diff
,<chr>,<date>,<dbl>,<dbl>,<dbl>
1,es0118a,2020-01-01,5.266250,8.064861,-34.70129
2,es0118a,2020-01-02,4.294583,10.435778,-58.84750
3,es0118a,2020-01-03,1.887500,13.732020,-86.25475
4,es0118a,2020-01-04,25.960000,13.684595,89.70236
5,es0118a,2020-01-05,19.327083,10.959504,76.34998
6,es0118a,2020-01-06,11.130833,9.711151,14.61910


In [10]:
write.csv(results, 
          paste("data/Analysis/predictions-", "lm", "-o3", ".csv", sep=""))